# Hello Object Detection

A very basic introduction to using object detection models with OpenVINO™.

The [horizontal-text-detection-0001](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/horizontal-text-detection-0001/README.md) model from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/) is used. It detects horizontal text in images and returns a blob of data in the shape of `[100, 5]`. Each detected text box is stored in the `[x_min, y_min, x_max, y_max, conf]` format, where the
`(x_min, y_min)` are the coordinates of the top left bounding box corner, `(x_max, y_max)` are the coordinates of the bottom right bounding box corner and `conf` is the confidence for the predicted class.

## Imports
[back to top ⬆️](#Table-of-contents:)


In [ ]:
import openvino as ov
import cv2
import numpy as np

import matplotlib.pyplot as plt #이미지 출력
from pathlib import Path #디렉토리 파일 불러오기

## Select inference device
select device from dropdown list for running inference using OpenVINO

In [ ]:
#device = device_widget()
#device

#위의 값을 대체
core=ov.Core()
options=core.available_devices
options

## Load the Model
[back to top ⬆️](#Table-of-contents:)


In [ ]:
model = core.read_model(model='model/horizontal-text-detection-0001.xml') #모델의 위치를 재설정
compiled_model = core.compile_model(model=model, device_name='CPU')
## device에서 선택했던 것을 사용하는 것

input_layer = compiled_model.input(0)
output_layer = compiled_model.output("boxes")

print(input_layer.shape)

## Load an Image
[back to top ⬆️](#Table-of-contents:)


In [ ]:
# Text detection models expect an image in BGR format.
image = cv2.imread('data/abc.jpg')

print(image.shape)

# N,C,H,W = batch size, number of channels, height, width.
N, C, H, W = input_layer.shape

# Resize the image to meet network expected input sizes.
resized_image = cv2.resize(image, (W, H))
# 이미지는 칼라면 뒤에 3이 붙는다. 흑백이면 0)
print(resized_image.shape)

# Reshape to the network input shape.
input_image = np.expand_dims(resized_image.transpose(2, 0, 1), 0)

print(input_image.shape)

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB));

## Do Inference
[back to top ⬆️](#Table-of-contents:)


In [ ]:
# Create an inference request.
boxes = compiled_model([input_image])[output_layer]
print(boxes)
print("----------------------")
# Remove zero only boxes.
boxes = boxes[~np.all(boxes == 0, axis=1)]
#이미지 박스 쳐져 있는 부분, 

## Visualize Results
[back to top ⬆️](#Table-of-contents:)


In [ ]:
# For each detection, the description is in the [x_min, y_min, x_max, y_max, conf] format:
# The image passed here is in BGR format with changed width and height. To display it in colors expected by matplotlib, use cvtColor function
def convert_result_to_image(bgr_image, resized_image, boxes, threshold=0.3, conf_labels=True): # threshold 30% 확률로 결정해라.
    # Define colors for boxes and descriptions.
    colors = {"red": (255, 0, 0), "green": (0, 255, 0)}
    #RGB이기에 255=R

    # Fetch the image shapes to calculate a ratio.
    (real_y, real_x), (resized_y, resized_x) = (
        bgr_image.shape[:2],     #517, 690
        resized_image.shape[:2], #704, 704
    )
    ratio_x, ratio_y = real_x / resized_x, real_y / resized_y

    # Convert the base image from BGR to RGB format.
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB) #bgr_image : 원래 이미지

    # Iterate through non-zero boxes.
    for box in boxes: #for은 반복문
        # Pick a confidence factor from the last place in an array.
        conf = box[-1]
        if conf > threshold:
            # Convert float to int and multiply corner position of each box by x and y ratio.
            # If the bounding box is found at the top of the image,
            # position the upper box bar little lower to make it visible on the image.
            (x_min, y_min, x_max, y_max) = [
                (int(max(corner_position * ratio_y, 10)) if idx % 2 else int(corner_position * ratio_x)) for idx, corner_position in enumerate(box[:-1])
            ]

            # Draw a box based on the position, parameters in rectangle function are: image, start_point, end_point, color, thickness.
            rgb_image = cv2.rectangle(rgb_image, (x_min, y_min), (x_max, y_max), colors["green"], 3) # 박스를 그릴떄, 왼쪽 최댓값, 오른쪽 최댓값, 3은 여기서 두께

            # Add text to the image based on position and confidence.
            # Parameters in text function are: image, text, bottom-left_corner_textfield, font, font_scale, color, thickness, line_type.
            if conf_labels:
                rgb_image = cv2.putText(
                    rgb_image,
                    f"{conf:.2f}",
                    (x_min, y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8,
                    colors["red"],
                    1,
                    cv2.LINE_AA,
                )

    return rgb_image

In [ ]:
plt.figure(figsize=(10, 6))
plt.axis("off") #죄표값 
plt.imshow(convert_result_to_image(image, resized_image, boxes, conf_labels=True));

# 6.배포

### Gradio

In [10]:
import openvino as ov
import cv2
import numpy as np

from pathlib import Path #디렉토리 파일 불러오기
import PIL
import gradio as gr

In [11]:
core=ov.Core()
options=core.available_devices
options

['CPU', 'GPU']

In [12]:
model = core.read_model(model='model/horizontal-text-detection-0001.xml') #모델의 위치를 재설정
compiled_model = core.compile_model(model=model, device_name='CPU')
## device에서 선택했던 것을 사용하는 것

input_layer = compiled_model.input(0)
output_layer = compiled_model.output("boxes")

print(input_layer.shape)

[1,3,704,704]


In [13]:
def preprocess(image):

    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    N, C, H, W = input_layer.shape
    resized_image = cv2.resize(image, (W, H))
    input_image = np.expand_dims(resized_image.transpose(2, 0, 1), 0)
    
    return input_image, resized_image

In [14]:
def convert_result_to_image(bgr_image, resized_image, boxes, threshold=0.3, conf_labels=True):
    # Define colors for boxes and descriptions.
    colors = {"red": (255, 0, 0), "green": (0, 255, 0)}

    # Fetch the image shapes to calculate a ratio.
    (real_y, real_x), (resized_y, resized_x) = (
        bgr_image.shape[:2],       # 517, 690
        resized_image.shape[:2],   #704, 704
    )
    ratio_x, ratio_y = real_x / resized_x, real_y / resized_y

    # Convert the base image from BGR to RGB format.
    # rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    # Iterate through non-zero boxes.
    for box in boxes:
        # Pick a confidence factor from the last place in an array.
        conf = box[-1]
        if conf > threshold:
            # Convert float to int and multiply corner position of each box by x and y ratio.
            # If the bounding box is found at the top of the image,
            # position the upper box bar little lower to make it visible on the image.
            (x_min, y_min, x_max, y_max) = [
                (int(max(corner_position * ratio_y, 10)) if idx % 2 else int(corner_position * ratio_x)) for idx, corner_position in enumerate(box[:-1])
            ]

            # Draw a box based on the position, parameters in rectangle function are: image, start_point, end_point, color, thickness.
            rgb_image = cv2.rectangle(bgr_image, (x_min, y_min), (x_max, y_max), colors["green"], 3)

            # Add text to the image based on position and confidence.
            # Parameters in text function are: image, text, bottom-left_corner_textfield, font, font_scale, color, thickness, line_type.
            if conf_labels:
                rgb_image = cv2.putText(
                    rgb_image,
                    f"{conf:.2f}",
                    (x_min, y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8,
                    colors["red"],
                    1,
                    cv2.LINE_AA,
                )

    return rgb_image

In [15]:
def predict_image(image):
    input_image, resized_image = preprocess(image)  #이미지 shape 전처리 
    
    boxes = compiled_model([input_image])[output_layer]
    boxes = boxes[~np.all(boxes == 0, axis=1)]
    canvas = convert_result_to_image(image, resized_image, boxes, conf_labels=True)
    
    return canvas

In [16]:
#image = cv2.imread('data/abc.jpg')
#plt.imshow(predict_image(image));

In [17]:
demo = gr.Interface(predict_image, gr.Image(), "image")
demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
